In [1]:
import pandas as pd
import gensim
import nltk
import re
import string

C:\Users\Christopher\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
ds_df = pd.read_csv("data\Data_Scientist_text_data.csv")

In [3]:
ds_df.head()

,Unnamed: 0,job_description,job_location,job_title,sponsored
0,0,"Given substantial growth and impact at Crowe, ...","Indianapolis, IN 46240",Data Scientist,True
1,2,"At EMC, you'll put your skills to good use as ...","Des Moines, IA",Data Scientist,True
2,3,Are you looking for a job in a growing industr...,"Lincoln, RI",Data Scientist,True
3,4,Ascensia Diabetes Care is a global specialist ...,"Valhalla, NY",Data Scientist,True
4,5,$90 - $95 an hourContractDescription: building...,"Durham, NC",Data Scientist w/ Watson Experience,True


In [7]:
ds_df.job_description = ds_df.job_description.str.lower()

In [5]:


# Create initial documents list

doc = list(ds_df.job_description)

# Remove punctuation, then tokenize documents
punc = re.compile( '[%s]' % re.escape( string.punctuation ) )
term_vec = [ ]

for d in doc:
    d = d.lower()
    d = punc.sub( '', d )
    term_vec.append( nltk.word_tokenize( d ) )

In [6]:
ds_df['clean_desc'] = term_vec

In [17]:
ds_df = ds_df.iloc[:,1:]

In [18]:
ds_df.to_csv("data_scientist_clean_text.csv", index=False)

In [16]:
stop_words = nltk.corpus.stopwords.words( 'english' )

for i in range( 0, len( term_vec ) ):
    term_list = [ ]

    for term in term_vec[ i ]:
        if term not in stop_words:
            term_list.append( term )

    term_vec[ i ] = term_list

In [17]:
# Stem terms
# Porter stem remaining terms

porter = nltk.stem.porter.PorterStemmer()

for i in range( 0, len( term_vec ) ):
    for j in range( 0, len( term_vec[ i ] ) ):
        term_vec[ i ][ j ] = porter.stem( term_vec[ i ][ j ] )

In [18]:
#  Convert term vectors into gensim dictionary

dict = gensim.corpora.Dictionary( term_vec )

corp = [ ]
for i in range( 0, len( term_vec ) ):
    corp.append( dict.doc2bow( term_vec[ i ] ) )

#  Create TFIDF vectors based on term vectors bag-of-word corpora

tfidf_model = gensim.models.TfidfModel( corp )

tfidf = [ ]
for i in range( 0, len( corp ) ):
    tfidf.append( tfidf_model[ corp[ i ] ] )

#  Create pairwise document similarity index

n = len( dict )
index = gensim.similarities.SparseMatrixSimilarity( tfidf_model[ corp ], num_features = n )

#  Print TFIDF vectors and pairwise similarity per document

for i in range( 0, len( tfidf ) ):
    s = 'Doc ' + str( i + 1 ) + ' TFIDF:'

    for j in range( 0, len( tfidf[ i ] ) ):
        s = s + ' (' + dict.get( tfidf[ i ][ j ][ 0 ] ) + ','
        s = s + ( '%.3f' % tfidf[ i ][ j ][ 1 ] ) + ')'


for i in range( 0, len( corp ) ):
    sim = index[ tfidf_model[ corp[ i ] ] ]


In [36]:
important_words = []

for i in range( 0, len( tfidf ) ):
    s = 'Doc ' + str( i + 1 ) + ' TFIDF:'

    for j in range( 0, len( tfidf[ i ] ) ):
        s = s + ' (' + dict.get( tfidf[ i ][ j ][ 0 ] ) + ','
        s = s + ( '%.3f' % tfidf[ i ][ j ][ 1 ] ) + ')'

    print s

AttributeError: 'list' object has no attribute 'shape'